# Turning CSV Into SQLITE
This notebook lays down the method for turning the downloaded CSV into an sqlite file, which is superior

The CSV file can be downloaded from [here](https://www.kaggle.com/datasets/wordsforthewise/lending-club/)

This notebook simply serves as a prototype, and the actual data is processed in the script `scripts/01 - csv_to_sqlite.py`.

## Imports and Definitions

In [1]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import sqlite3       # For SQLite database management
import numpy as np   # For numerical operations
import datetime      # For handling date and time
import time          # For time-related tasks

def convert_to_unix_time(date_str):
    """
    Convert a date string to Unix time.
    :param date_str: A string representing a date in 'Mon-Year' format (e.g., 'Dec-2015').
    :return: Unix time as an integer.
    """
    # Parse the date string to a datetime object, assuming the first day of the month
    dt = datetime.datetime.strptime(date_str, "%b-%Y")

    # Convert the datetime object to Unix time (seconds since Jan 1, 1970)
    unix_time = int(time.mktime(dt.timetuple()))

    return unix_time

def analyze_column(column):
    """
    Analyze the data type of a column in a DataFrame.
    :param column: A pandas Series representing a DataFrame column.
    :return: A string indicating the data type of the column.
    """
    # Check if all elements in the column are strings
    if all(isinstance(x, str) for x in column):
        # Check if all strings are either numeric or 'nan'
        if all(x.replace('.', '', 1).isdigit() or x.lower() == 'nan' for x in column):
            return 'REVIEW_TEXT'
        else:
            return 'TEXT'
    
    # If not all elements are strings, check other data types
    else:
        # Check if all elements are floats
        if all(isinstance(x, float) for x in column):
            # Check if floats can be converted to integers without loss of information
            if not all(x.is_integer() for x in column):
                return 'FLOAT'
            else:
                return 'REVIEW_FLOAT'
        # Check if all elements are integers
        elif all(isinstance(x, (int, np.integer)) for x in column):
            return 'INT'
        # If none of the above, return 'REVIEW' for further examination
        else:
            return 'REVIEW'

# Path to the CSV file containing loan data
csv_file = '../data/accepted_2007_to_2018Q4.csv'
# Path to the sqlite to be created/edited
sqlite_file = '../data/All_Lending_Club_Loan_2007_2018.sqlite'

## Loading and Cleaning Data

In [2]:
# Loading the loan data file
print('Loading File')
df = pd.read_csv(csv_file, low_memory=False)  # Load the CSV file into a pandas DataFrame

# Extracting metadata from the last two rows of the dataset
print('Loading Metadata')
metadata = df.iloc[-2:]['id'].values  # Store the last two rows' 'id' values as metadata
df = df.iloc[:-2]  # Remove the last two rows from the DataFrame

# Converting date columns to Unix time format
print('Creating UNIX columns for dates')
dates_columns = ['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d',
                 'sec_app_earliest_cr_line', 'hardship_start_date', 'hardship_end_date',
                 'payment_plan_start_date', 'settlement_date', 'debt_settlement_flag_date']
for column in dates_columns:
    # Convert each date column to Unix time, handling NaN values appropriately
    df[f'{column}_unix'] = df[column].astype('str').apply(lambda x: convert_to_unix_time(x) if x != 'nan' else pd.NA)

# Creating new derived columns for analysis
print('Creating other interesting columns')
df['term_months'] = df['term'].apply(lambda x: int(x[1:3]))  # Convert loan term to integer months
# Convert employment length to integer years, handling special cases and NaNs
df['emp_length_years'] = df['emp_length'].astype('str').apply(lambda x: int(x.split(' ')[0].replace('+','')) if x != '< 1 year' and x != 'nan' else pd.NA)
df['loan_id'] = df['url'].apply(lambda x: int(x.split('loan_id=')[-1]))  # Extract loan ID from URL
df['id'] = range(len(df))  # Assign a new sequential ID to each row

# Dropping original columns that are no longer needed or have been transformed
print("Dropping proper columns")
drop_columns = ['member_id', 'term', 'emp_length'] + dates_columns
for column in drop_columns:
    df.drop(column, axis='columns', inplace=True)

# Filling missing values in specific columns with 0
print("Filling proper columns")
fillna_columns = ['tot_coll_amt', 'tot_cur_bal']
for column in fillna_columns:
    df[column].fillna(0., inplace=True)

# Casting columns to their appropriate data types
print("Casting columns in proper types")
# Convert specified columns to string, handling NaN values
str_columns = ['emp_title', 'desc', 'title', 'verification_status_joint', 'hardship_type',
               'hardship_reason', 'hardship_status', 'hardship_loan_status', 'settlement_status']
for column in str_columns:
    df[column] = df[column].astype('str').apply(lambda x: x if x != 'nan' else pd.NA)

# Convert specified columns to integer
int_columns = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'annual_inc', 'delinq_2yrs',
               'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec',
               'revol_bal', 'total_acc', 'last_fico_range_low', 'last_fico_range_high',
               'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'tot_coll_amt',
               'tot_cur_bal', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']
for column in int_columns:
    df[column] = df[column].astype('int')

# Convert specified columns to float
float_columns = ['int_rate', 'installment', 'dti']
for column in float_columns:
    df[column] = df[column].astype('float')

print("Adjusting all missing values to pd.NA")
for column in df.columns:
    df[column] = df[column].replace([None, np.nan], pd.NA)
# Converting metadata to its own dataframe
metadata = pd.DataFrame({s.split(': ')[0]:[int(s.split(': ')[-1])] for s in metadata})
print("Done")  # Indicate completion of data processing

Loading File
Loading Metadata
Dropping NAs
Creating UNIX columns for dates
Creating other interesting columns
Dropping proper columns
Filling proper columns
Casting columns in proper types
Adjusting all missing values to pd.NA
Done


In [3]:
column_types = {}
for column in df.columns:
    col_type = str(df[column].dtype)
    if col_type == 'int64':
        column_types[column] = 'INT'
    elif col_type == 'float64':
        column_types[column] = 'REAL'
    else:
        column_types[column] = 'TEXT' 

table_descriptions = {
    'loans_data':f'Data for all the loans available in the database, downloaded as a csv, cleaned, and put into a sqlite file on 2023-12-06. Original download from https://www.kaggle.com/code/pavlofesenko/minimizing-risks-for-loan-investments. Contains columns: {", ".join(df.columns)}.',
    'metadata':f'Metadata provided on the downloaded data, providing the total amount funded in different policy codes. Contains columns: {", ".join(metadata.columns)}.',
    'descriptions':f'A table containing a written description of each available table, and column. Contains columns: loans_data, metadata, descriptions, {", ".join(df.columns)}, {", ".join(metadata.columns)}.'
}

metadata_columns_description = {
    'Total amount funded in policy code 1': 'Contains the total amount, in dollars, funded in loans with policy code 1.',
    'Total amount funded in policy code 2': 'Contains the total amount, in dollars, funded in loans with policy code 2.'
}

loans_data_columns_description = {
    'id':'A row identifyer for the loans table. Each id is unique for each row.',
    'loan_amnt': 'The listed amount of the loan applied for by the borrower.',
    'funded_amnt': 'The total amount committed to the loan by the Lending Club.',
    'funded_amnt_inv': 'The total amount committed by investors for the loan.',
    'int_rate': 'Interest rate of the loan.',
    'installment': 'The monthly payment owed by the borrower if the loan originates.',
    'grade': 'Lending Club assigned loan grade.',
    'sub_grade': 'Lending Club assigned sub-grade.',
    'emp_title': 'The job title supplied by the borrower when applying for the loan.',
    'home_ownership': 'The home ownership status provided by the borrower during registration. Values are: RENT, OWN, MORTGAGE, OTHER.',
    'annual_inc': 'The self-reported annual income provided by the borrower during registration.',
    'verification_status': 'Indicates if income was verified by Lending Club.',
    'loan_status': 'Current status of the loan.',
    'pymnt_plan': 'Indicates if a payment plan has been put in place for the loan.',
    'url': 'URL for the Lending Club page with detailed information about the loan.',
    'desc': 'Loan description provided by the borrower.',
    'purpose': 'A category provided by the borrower for the loan request.',
    'title': 'The loan title provided by the borrower.',
    'zip_code': 'The first 3 numbers of the zip code provided by the borrower in the loan application.',
    'addr_state': 'The state provided by the borrower in the loan application.',
    'dti': 'Debt-to-income ratio calculated using the borrower’s total monthly debt payments divided by their self-reported monthly income.',
    'delinq_2yrs': 'The number of 30+ days delinquencies the borrower has had in the past 2 years.',
    'earliest_cr_line': 'The month and year the borrower’s earliest reported credit line was opened.',
    'fico_range_low': 'The lower boundary range of the borrower’s FICO score at the time of loan origination.',
    'fico_range_high': 'The upper boundary range of the borrower’s FICO score at the time of loan origination.',
    'inq_last_6mths': 'The number of credit inquiries the borrower has had in the last 6 months (excluding auto and mortgage inquiries).',
    'mths_since_last_delinq': 'The number of months since the borrower’s last delinquency.',
    'mths_since_last_record': 'The number of months since the last public record.',
    'open_acc': 'The number of open credit lines in the borrower’s credit file.',
    'pub_rec': 'Number of derogatory public records, such as bankruptcy filings, tax liens, or judgments.',
    'revol_bal': 'The total amount of credit revolving balances.',
    'revol_util': 'Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.',
    'total_acc': 'The total number of credit lines currently in the borrower’s credit file.',
    'initial_list_status': 'The initial listing status of the loan. Possible values are – W, F.',
    'out_prncp': 'The remaining outstanding principal for total amount funded.',
    'out_prncp_inv': 'The remaining outstanding principal for portion of total amount funded by investors.',
    'total_pymnt': 'Payments received to date for total amount funded.',
    'total_pymnt_inv': 'Payments received to date for portion of total amount funded by investors.',
    'total_rec_prncp': 'Principal received to date.',
    'total_rec_int': 'Interest received to date.',
    'total_rec_late_fee': 'Late fees received to date.',
    'recoveries': 'Post charge off gross recovery.',
    'collection_recovery_fee': 'Post charge off collection fee.',
    'last_pymnt_amnt': 'Last total payment amount received.',
    'last_fico_range_high': 'The upper boundary range of the borrower’s last FICO score.',
    'last_fico_range_low': 'The lower boundary range of the borrower’s last FICO score.',
    'collections_12_mths_ex_med': 'Number of collections in 12 months excluding medical collections.',
    'mths_since_last_major_derog': 'Months since most recent 90-day or worse rating.',
    'policy_code': 'Publicly available policy_code=1, new products not publicly available policy_code=2.',
    'application_type': 'Indicates whether the loan is an individual application or a joint application with two co-borrowers.',
    'annual_inc_joint': 'The combined annual income reported by the co-borrowers on a joint application.',
    'dti_joint': 'The debt-to-income ratio calculated using the total monthly debt payments of the co-borrowers, divided by the combined self-reported monthly income of the co-borrowers, on a joint application.',
    'verification_status_joint': 'Indicates if the co-borrowers’ joint income was verified by Lending Club.',
    'acc_now_delinq': 'The number of accounts on which the borrower is now delinquent.',
    'tot_coll_amt': 'Total collection amounts ever owed.',
    'tot_cur_bal': 'Total current balance of all accounts.',
    'open_acc_6m': 'Number of open trades in the last 6 months.',
    'open_act_il': 'Number of currently active installment trades.',
    'open_il_12m': 'Number of installment accounts opened in past 12 months.',
    'open_il_24m': 'Number of installment accounts opened in past 24 months.',
    'mths_since_rcnt_il': 'Months since the most recent installment account opened.',
    'total_bal_il': 'Total current balance of all installment accounts.',
    'il_util': 'Ratio of total current balance to high credit/credit limit on all installment accounts.',
    'open_rv_12m': 'Number of revolving trades opened in past 12 months.',
    'open_rv_24m': 'Number of revolving trades opened in past 24 months.',
    'max_bal_bc': 'Maximum current balance owed on all revolving accounts.',
    'all_util': 'Balance to credit limit on all trades.',
    'total_rev_hi_lim': 'Total revolving high credit/credit limit.',
    'inq_fi': 'Number of personal finance inquiries.',
    'total_cu_tl': 'Number of finance trades.',
    'inq_last_12m': 'Number of credit inquiries in the past 12 months.',
    'acc_open_past_24mths': 'Number of accounts opened in past 24 months.',
    'avg_cur_bal': 'Average current balance of all accounts.',
    'bc_open_to_buy': 'Total open to buy on revolving bankcards.',
    'bc_util': 'Ratio of total current balance to high credit/credit limit for all bankcard accounts.',
    'chargeoff_within_12_mths': 'Number of charge-offs within  last 12 months.',
    'delinq_amnt': 'The past-due amount owed for the accounts on which the borrower is now delinquent.',
    'mo_sin_old_il_acct': 'Months since oldest bank installment account opened.',
    'mo_sin_old_rev_tl_op': 'Months since oldest revolving account opened.',
    'mo_sin_rcnt_rev_tl_op': 'Months since most recent revolving account opened.',
    'mo_sin_rcnt_tl': 'Months since most recent account opened.',
    'mort_acc': 'Number of mortgage accounts.',
    'mths_since_recent_bc': 'Months since most recent bankcard account opened.',
    'mths_since_recent_bc_dlq': 'Months since most recent bankcard delinquency.',
    'mths_since_recent_inq': 'Months since most recent inquiry.',
    'mths_since_recent_revol_delinq': 'Months since most recent revolving delinquency.',
    'num_accts_ever_120_pd': 'Number of accounts ever 120 or more days past due.',
    'num_actv_bc_tl': 'Number of currently active bankcard accounts.',
    'num_actv_rev_tl': 'Number of currently active revolving trades.',
    'num_bc_sats': 'Number of satisfactory bankcard accounts.',
    'num_bc_tl': 'Number of bankcard accounts (credit cards issued by banks).',
    'num_il_tl': 'Number of installment loan accounts (loans with fixed payments and a set maturity date).',
    'num_op_rev_tl': 'Number of open revolving accounts (credit cards and lines of credit).',
    'num_rev_accts': 'Number of revolving accounts.',
    'num_rev_tl_bal_gt_0': 'Number of revolving trades with balance greater than zero.',
    'num_sats': 'Number of satisfactory accounts.',
    'num_tl_120dpd_2m': 'Number of accounts currently 120 days past due (updated in past 2 months).',
    'num_tl_30dpd': 'Number of accounts currently 30 days past due.',
    'num_tl_90g_dpd_24m': 'Number of accounts 90 or more days past due in last 24 months.',
    'num_tl_op_past_12m': 'Number of accounts opened in past 12 months.',
    'pct_tl_nvr_dlq': 'Percentage of accounts never delinquent.',
    'percent_bc_gt_75': 'Percentage of bankcard accounts with credit utilization greater than 75%.',
    'pub_rec_bankruptcies': 'Number of public record bankruptcies.',
    'tax_liens': 'Number of tax liens.',
    'tot_hi_cred_lim': 'Total high credit/credit limit (the total of the maximum amount of credit extended on all accounts).',
    'total_bal_ex_mort': 'Total credit balance excluding mortgage.',
    'total_bc_limit': 'Total bankcard (credit card) limit.',
    'total_il_high_credit_limit': 'Total installment high credit/credit limit.',
    'revol_bal_joint': 'Total revolving balance on joint accounts.',
    'sec_app_fico_range_low': 'FICO range (low) for the secondary applicant.',
    'sec_app_fico_range_high': 'FICO range (high) for the secondary applicant.',
    'sec_app_inq_last_6mths': 'Number of credit inquiries for the secondary applicant in the last 6 months.',
    'sec_app_mort_acc': 'Number of mortgage accounts held by the secondary applicant.',
    'sec_app_open_acc': 'Number of open credit lines in the secondary applicant’s credit file.',
    'sec_app_revol_util': 'Revolving line utilization rate, or the amount of credit the secondary applicant is using relative to all available revolving credit.',
    'sec_app_open_act_il': 'Number of currently active installment trades for the secondary applicant.',
    'sec_app_num_rev_accts': 'Number of revolving accounts held by the secondary applicant.',
    'sec_app_chargeoff_within_12_mths': 'Number of charge-offs within the last 12 months for the secondary applicant.',
    'sec_app_collections_12_mths_ex_med': 'Number of collections in 12 months excluding medical collections for the secondary applicant.',
    'sec_app_mths_since_last_major_derog': 'Months since the secondary applicant’s last major derogatory event.',
    'hardship_flag': 'Flags whether the borrower is on a hardship plan.',
    'hardship_type': 'Type of hardship plan offered to the borrower, if any.',
    'hardship_reason': 'Reason provided by the borrower for the hardship event.',
    'hardship_status': 'Current status of the hardship plan.',
    'deferral_term': 'The number of months that the borrower is expected to pay less than the contractual monthly payment amount due to a hardship plan.',
    'hardship_amount': 'The amount of the monthly payment that the borrower must pay while under a hardship plan.',
    'hardship_length': 'The length of the hardship plan in months.',
    'hardship_dpd': 'Number of days past due as of the hardship plan start date.',
    'hardship_loan_status': 'Loan status as of the hardship plan start date.',
    'orig_projected_additional_accrued_interest': 'The original estimated total amount of additional interest the borrower will pay as a result of the hardship plan.',
    'hardship_payoff_balance_amount': 'The balance amount at the time the borrower entered into a hardship plan.',
    'hardship_last_payment_amount': 'The last payment amount made by the borrower before entering the hardship plan.',
    'disbursement_method': 'Method by which the borrower receives their loan. Possible values are \'cash\', \'directpay\', etc.',
    'debt_settlement_flag': 'Indicates whether the borrower, who has charged-off, is working with a debt settlement company.',
    'settlement_status': 'The status of the borrower’s settlement plan. Possible values are \'active\', \'completed\', \'broken\', etc.',
    'settlement_amount': 'The amount that the borrower has agreed to pay in a settlement plan.',
    'settlement_percentage': 'The percentage of the unpaid principal balance the borrower has agreed to pay in a settlement plan.',
    'settlement_term': 'The number of months the borrower will be on the settlement plan.',
    'issue_d_unix': 'The date when the loan was funded, converted to Unix time.',
    'last_pymnt_d_unix': 'The date of the last payment received, converted to Unix time.',
    'next_pymnt_d_unix': 'The date of the next scheduled payment, converted to Unix time.',
    'last_credit_pull_d_unix': 'The most recent date Lending Club pulled credit for this loan, converted to Unix time.',
    'sec_app_earliest_cr_line_unix': 'Earliest credit line date for the secondary applicant, converted to Unix time.',
    'hardship_start_date_unix': 'The start date of the hardship plan, converted to Unix time.',
    'hardship_end_date_unix': 'The end date of the hardship plan, converted to Unix time.',
    'payment_plan_start_date_unix': 'The start date of the payment plan, converted to Unix time.',
    'settlement_date_unix': 'The date that the borrower agrees to the settlement plan, converted to Unix time.',
    'debt_settlement_flag_date_unix': 'The date that the borrower\'s debt settlement flag was set, converted to Unix time.',
    'term_months': 'The length of the loan term in months.',
    'emp_length_years': 'The borrower’s length of employment in years.',
    'loan_id':'Identification nunber of the loan according to the url associated'    
}

descriptions_columns_description = {
    'name': 'The name of the element to be described',
    'type': 'The type of the element, table or column',
    'location': 'The table where the element is located (root if it\'s a table)',
    'description': 'A description of the element'
}

descriptions = {'name':[], 'type':[], 'location':[], 'description':[]}

for key, value in table_descriptions.items():
    descriptions['name'] += [key]
    descriptions['type'] += ['table']
    descriptions['location'] += ['root']
    descriptions['description'] += [value]
    
for key, value in metadata_columns_description.items():
    descriptions['name'] += [key]
    descriptions['type'] += ['column']
    descriptions['location'] += ['metadata']
    descriptions['description'] += [value]

for key, value in loans_data_columns_description.items():
    descriptions['name'] += [key]
    descriptions['type'] += ['column']
    descriptions['location'] += ['loans_data']
    descriptions['description'] += [value]
    
for key, value in descriptions_columns_description.items():
    descriptions['name'] += [key]
    descriptions['type'] += ['column']
    descriptions['location'] += ['descriptions']
    descriptions['description'] += [value]

descriptions = pd.DataFrame(descriptions)

In [4]:
# Create a SQLite database
print("Connecting to Database")
conn = sqlite3.connect(sqlite_file)

print("Creating Queries")
# Query to delete loans_data if it exists
drop_loans_data_query = 'DROP TABLE IF EXISTS loans_data'
# Query to delete metadata if it exists
drop_metadata_query = 'DROP TABLE IF EXISTS metadata'
# Query to delete descriptions if it exists
drop_descriptions_query = 'DROP TABLE IF EXISTS descriptions'

# Query to create a table for the loans data
create_loans_data_table_query = 'CREATE TABLE loans_data (' + ', '.join([f"\"{col}\" {col_type}" for col, col_type in column_types.items()]) + ')'
# Query to create a table for the metadata
create_metadata_table_query = 'CREATE TABLE metadata (' + ', '.join([f"\"{col}\" TEXT" for col in metadata.columns]) + ')'
# Query to create a table for the descriptions
create_descriptions_table_query = 'CREATE TABLE descriptions (' + ', '.join([f"\"{col}\" TEXT" for col in descriptions.columns]) + ')'

print("Dropping old tables and creating new ones")
# Drops and creates the tables
conn.execute(drop_loans_data_query)
conn.execute(create_loans_data_table_query)
conn.execute(drop_metadata_query)
conn.execute(create_metadata_table_query)
conn.execute(drop_descriptions_query)
conn.execute(create_descriptions_table_query)

print("Loading data into tables")
# Insert data from DataFrame to the SQLite table
df.to_sql('loans_data', conn, if_exists='replace', index=False)
metadata.to_sql('metadata', conn, if_exists='replace', index=False)
descriptions.to_sql('descriptions', conn, if_exists='replace', index=False)

conn.close()
print("Done")

Connecting to Database
Creating Queries
Dropping old tables and creating new ones
Loading data into tables
Done


In [5]:
# Connect to the SQLite database
conn = sqlite3.connect(sqlite_file)

# SQL query to select all data from the loans_data table
query = "SELECT * FROM loans_data LIMIT 100;"

# Read the query results into a pandas DataFrame
df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

df

,id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,sub_grade,emp_title,home_ownership,...,last_credit_pull_d_unix,sec_app_earliest_cr_line_unix,hardship_start_date_unix,hardship_end_date_unix,payment_plan_start_date_unix,settlement_date_unix,debt_settlement_flag_date_unix,term_months,emp_length_years,loan_id
0,0,3600,3600,3600,13.99,123.03,C,C4,leadman,MORTGAGE,...,1551420000,None,None,None,None,NaN,NaN,36,10.0,68407277
1,1,24700,24700,24700,11.99,820.28,C,C1,Engineer,MORTGAGE,...,1551420000,None,None,None,None,NaN,NaN,36,10.0,68355089
2,2,20000,20000,20000,10.78,432.66,B,B4,truck driver,MORTGAGE,...,1551420000,None,None,None,None,NaN,NaN,60,10.0,68341763
3,3,35000,35000,35000,14.85,829.90,C,C5,Information Systems Officer,MORTGAGE,...,1551420000,None,None,None,None,NaN,NaN,60,10.0,66310712
4,4,10400,10400,10400,22.45,289.91,F,F1,Contract Specialist,MORTGAGE,...,1519884000,None,None,None,None,NaN,NaN,60,3.0,68476807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,8000,8000,8000,10.78,261.08,B,B4,Counselor,MORTGAGE,...,1551420000,None,None,None,None,NaN,NaN,36,NaN,68537564
96,96,12000,12000,12000,10.78,391.62,B,B4,Salesman,RENT,...,1549000800,None,None,None,None,NaN,NaN,36,3.0,68446784
97,97,18000,18000,18000,7.49,559.83,A,A4,Systems Support Engineer,MORTGAGE,...,1549000800,None,None,None,None,NaN,NaN,36,5.0,68376533
98,98,3600,3600,3600,11.48,118.68,B,B5,Systems Administrator,MORTGAGE,...,1546322400,None,None,None,None,NaN,NaN,36,NaN,68476714
